### Importing required libraries

In [1]:
import os
import sys
sys.path.append("..")

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.login_utils import log_llm_call
from utils.router import pick_model, should_use_reasoning_model


### 01.Zero shot


In [2]:
# Zero-shot classification
prompt_text, spec = render(
    'zero_shot.v1',
    role='sentiment classifier',
    instruction='Classify the sentiment as positive, negative, or neutral',
    constraints='Return only the label',
    format='Single word'
)

model = pick_model('google','general')
llm_client = LLMClient('google',model)

test_text = 'This product is amazing! Best purchase ever.'
messages = [{'role': 'user', 'content': f"{prompt_text}\n\nText: {test_text}"}]

response = llm_client.chat(messages,temperature=0.0)
print('Zero-shot result:', response['text'])
log_llm_call('openai', model, 'zero_shot', response['latency_ms'], response['usage'])

Zero-shot result: Positive


### 02.Few shot


In [5]:
# Few-shot with comprehensive examples
examples = """
Example 1:
Review: I'm really happy with the product but it's bad!
Sentiment: negative
Explanation: User says product is bad but they are happy. When there's a conflict between 
user emotion and product quality, prioritize the product quality statement. 
Therefore, sentiment is negative.

Example 2:
Review: I'm really unhappy with the product but it's amazing!
Sentiment: positive
Explanation: User says product is amazing but they are unhappy. Prioritizing product 
quality over user emotion, the sentiment is positive.

Example 3:
Review: I'm really happy with the product! It's amazing!
Sentiment: positive
Explanation: Both user emotion and product quality are positive. Clear positive sentiment.

Example 4:
Review: I'm really unhappy with the product! It's terrible!
Sentiment: negative
Explanation: Both user emotion and product quality are negative. Clear negative sentiment.

Example 5:
Review: The product is okay, but it's not great.
Sentiment: neutral
Explanation: User expresses ambivalence. Product quality is neither strongly positive nor negative.

Example 6:
Review: I'm not sure about the product yet.
Sentiment: neutral
Explanation: User is uncertain and hasn't formed a clear opinion about product quality.
"""
model = pick_model('google','strong')
llm_client = LLMClient('google',model)

test_text_nuanced = "It seems nice but it's not for me"
prompt_text, spec = render(
    'few_shot.v1',
    role='sentiment classifier',
    examples=examples,
    query=f'Review: {test_text_nuanced}',
    constraints='Follow the pattern in examples: provide sentiment and brief explanation',
    format='Sentiment: {{sentiment}}\n\nExplanation: {{explanation}}'
)

messages = [{'role': 'user', 'content': prompt_text}]
response = llm_client.chat(messages, temperature=0.2)
print('Few-shot result (nuanced):')
print(response["text"])
log_llm_call('gemini', model, 'few_shot', response['latency_ms'], response['usage'])

Few-shot result (nuanced):
Sentiment: neutral

Explanation: The review contains conflicting statements. The user acknowledges a positive quality ("nice") but immediately negates the utility or personal fit ("not for me"). This ambivalence results in a mixed or neutral overall sentiment.


### 03.COT

In [11]:
model = pick_model('google','strong')
llm_client = LLMClient('google',model)

problem = """A car travels 100 miles in 2 hours. 

Question 1: What is the average speed of the car?
Question 2: If the car stopped for 40 minutes during this 2-hour journey, 
what was the average speed during the actual driving time?

Important: The 2 hours already includes the 40-minute stop."""

# Additional guidance
instruction = """Solve the following problem step by step.
1. First identify whether the car travelled the entire time without stopping or not.
2. If car stopped for x minutes and overall travelled for y hours, the actual driving duration is y-x.
3. If stopping time x is mentioned, do not add it to the travel duration because it's already included.
   So actual travel time is y (total time) - x (stopping time).
4. If car travelled the entire time without stopping, then average speed is distance / y.
5. If car stopped for x minutes, then average speed during driving is distance / (y - x)."""

prompt_text, spec = render(
    'cot_reasoning.v1',
    role='math tutor',
    problem=problem
)

# Combine problem with instruction (as done in live class)
full_prompt = f"""text: {prompt_text}

instruction: {instruction}"""

messages = [{'role':'user', 'content': full_prompt}]
response = llm_client.chat(messages, temperature=spec.temperature, max_tokens=spec.max_tokens)

print('CoT Response (Travel Time with Break):')
print('=' * 80)
print(response["text"])
print('=' * 80)
log_llm_call('google', model, 'cot', response['latency_ms'], response['usage'])

CoT Response (Travel Time with Break):
**Reasoning Steps Outline:**

1.  **Question 1 (Overall Speed):** Calculate the average speed by dividing the total distance (100 miles) by the total time elapsed (2 hours).
2.  **Unit Conversion:** Convert the stopping time (40 minutes) into hours ($40/60$ hours).
3.  **Question 2 (Actual Driving Time):** Determine the actual driving duration by subtracting the stopping time (in hours) from the total elapsed time (2 hours).
4.  **Question 2 (Driving Speed):** Calculate the average speed during the driving time by dividing the total distance (100 miles) by the actual driving duration.

---

### Step-by-Step Solution

**Given Data:**
*   Total Distance ($D$): 100 miles
*   Total Time Elapsed ($T_{total}$): 2 hours
*   Stopping Time ($T_{stop}$): 40 minutes

**Question 1: What is the average speed of the car?**

This calculation uses the total time elapsed, including the stop.
$$
\text{Average Speed} = \frac{\text{Total Distance}}{\text{Total Time E

### 04.TOT